In [1]:
%%capture
!pip install datasets llama-index==0.10.37 llama-index-embeddings-cohere qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

In [2]:
%%capture
!pip3 install datasets llama-index-embeddings-fastembed llama-index-llms-mistralai

In [3]:
%%capture
!pip install llama-index-llms-ollama llama-index-embeddings-cohere

Note: you should install the following packages to your environment:

`pip install datasets`

`pip install llama-index-embeddings-fastembed`

`pip install llama-index-llms-mistralai`

In [4]:
import os
import sys
from datasets import Dataset
from dotenv import load_dotenv
from getpass import getpass
import nest_asyncio
from IPython.display import Markdown, display


nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, create_index

In [5]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [6]:
# Remove OpenAI API key since we're not using OpenAI
# OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [7]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [8]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [9]:
from llama_index.core.settings import Settings
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama

from utils import setup_llm, setup_embed_model, setup_vector_store, create_index

setup_llm(provider="ollama", model="", api_key="")
setup_embed_model(provider="cohere", api_key=CO_API_KEY)

# Debug: Check if embedding model is set correctly
print(f"Embedding model type: {type(Settings.embed_model)}")
print(f"Embedding model: {Settings.embed_model}")

COLLECTION_NAME = "it_can_be_done"

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Explicitly pass the embed_model to ensure it's used
index = create_index(
    from_where="vector_store", 
    vector_store=vector_store, 
    storage_context=storage_context,
    embed_model=Settings.embed_model
)

Embedding model type: <class 'llama_index.embeddings.cohere.base.CohereEmbedding'>
Embedding model: model_name='embed-english-v3.0' embed_batch_size=10 callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x30cae9590> num_workers=None embeddings_cache=None api_key='D837umjWnWmrAcfabry3it8bn3ZZs74TOLi73Zqp' base_url=None truncate='END' input_type=None embedding_type='float'


# Default Prompt Templates

In [10]:
from utils import display_prompt_dict, create_query_engine

In [11]:
query_engine = create_query_engine(
    index=index,
    similarity_top_k=3, 
    mode="query",
    return_sources=True
    )

display_prompt_dict(query_engine.get_prompts())

 **Prompt Key**: response_synthesizer:text_qa_template
**Text:**
```
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 
```

**Prompt Key**: response_synthesizer:refine_template
**Text:**
```
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
```



In [12]:
from llama_index.core.prompts import PromptTemplate

custom_prompt = """You are an assistant for question-answering tasks related to \
motivational poetry. Your must reponse with an original Haiku style poem.

Use the following pieces of retrieved context to answer the user's query:

---------------------\n
{context_str}\n
---------------------\n

Query: {query_str}
"""

custom_prompt_template = PromptTemplate(custom_prompt)

In [13]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": custom_prompt_template}
)

In [14]:
display_prompt_dict(query_engine.get_prompts())

 **Prompt Key**: response_synthesizer:text_qa_template
**Text:**
```
You are an assistant for question-answering tasks related to motivational poetry. Your must reponse with an original Haiku style poem.

Use the following pieces of retrieved context to answer the user's query:

---------------------

{context_str}

---------------------


Query: {query_str}

```

**Prompt Key**: response_synthesizer:refine_template
**Text:**
```
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
```



In [15]:
from llama_index.core.workflow import Workflow, step, StartEvent, StopEvent

class QueryEvent(StartEvent):
    input: str

class ResponseEvent(StopEvent):
    result: str

class QueryWorkflow(Workflow):
    @step
    async def run_query(self, ev: QueryEvent) -> ResponseEvent:
        response = query_engine.query(ev.input)
        return ResponseEvent(result=str(response))

workflow = QueryWorkflow()

/var/folders/5p/trf91_p17bg3d0t9l5pl9kdm0000gn/T/ipykernel_83703/4238623573.py:6: UserWarning: Field name "result" in "ResponseEvent" shadows an attribute in parent "StopEvent"
  class ResponseEvent(StopEvent):


In [16]:
import asyncio

async def run_workflow():
    response = await workflow.run(input="If you keep your head when all around you are losing their cool and blaming it on you")
    return response

result = asyncio.run(run_workflow())

In [17]:
print(result)

Head held high amidst storm,
Blame dances like leaves in wind.
Stay true, hold on tight.


# Response Synthesizers

The Llama Index [documentation](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/) has a lot of detail regarding each of the response sythensizers. 

In [18]:
from llama_index.core.response_synthesizers import ResponseMode

In [19]:
dir(ResponseMode)

['ACCUMULATE',
 'COMPACT',
 'COMPACT_ACCUMULATE',
 'CONTEXT_ONLY',
 'GENERATION',
 'NO_TEXT',
 'REFINE',
 'SIMPLE_SUMMARIZE',
 'TREE_SUMMARIZE',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__members__',
 '__mod__',
 '__module__',
 '__mul__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'l

# Response modes

In LlamaIndex, [response modes](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/) are used to determine how the system should process and return the results of a query.  Each response mode is designed to handle different types of queries and use cases, providing flexibility and customization in how you interact with your data.



#### ⚗️ Refine 

Refine is an iterative method to generate a response. 

Initially, we use the context in the first node and the query to create a basic answer. Then, we refine this answer by inputting it, along with the query and context of the second node, into a "refine prompt" to generate an improved answer. 

This refinement process continues through N-1 nodes, with N being the total number of nodes. It makes a separate LLM call per Node/retrieved chunk. This mode is good for generating more detailed answers.

#### 🤏 Compact

Compact and refine mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then refine answers across them. This mode is faster than refine since we make fewer calls to the LLM. 

This mode is useful when you want to reduce the number of LLM calls while still refining the answer.


#### 📝 Simple summarize

Merge all text chunks into one and make a large language model call. The call will fail if the merged text chunk exceeds the context window size.

It's good for quick summarization purposes, but may lose detail due to truncation.

#### 🌴 Tree summarize

Construct a tree index for the candidate nodes in a bottom-up manner then use a summary prompt based on the query. Return the root node as the final response. When this mode is set, the system is instructed to iterate through many, if not all, documents in order to synthesize an answer, which can lead to better summarization results. 

This mode is particularly useful for summarization queries, where the goal is to provide a comprehensive summary of a collection of text or a specific topic.

#### 🤖 Generation

Ignore context, just use LLM to generate a response.

#### ❌ No text

This mode only runs the retriever to fetch the nodes that would have been sent to the LLM, without actually synthesizing a final response. The nodes can then be inspected by checking `response.source_nodes`.

#### 📏 Accumulate

This mode applies the query to each text chunk while accumulating the responses into an array. It returns a concatenated string of all responses. 

This mode is good for when you need to run the same query separately against each text chunk.

#### Compact accumulate

In the compact and accumulate mode, text chunks are combined into larger chunks to utilize the context window better. Answers are then accumulated for each chunk and returned as a concatenation. This mode is faster than accumulate as it reduces calls to the LLM.

In [20]:
from llama_index.core import get_response_synthesizer
from llama_index.core.workflow import Workflow, step, StartEvent, StopEvent

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = create_query_engine(
    index,
    mode="query",
    response_synthesizer = response_synthesizer
    )

class QueryEvent(StartEvent):
    input: str

class ResponseEvent(StopEvent):
    result: str

class SynthWorkflow(Workflow):
    @step
    async def synth(self, ev: QueryEvent) -> ResponseEvent:
        response = query_engine.query(ev.input)
        return ResponseEvent(result=str(response))

workflow = SynthWorkflow()

import asyncio

async def run_workflow():
    response = await workflow.run(input="What do the poems teach about one should think about success and failure?")
    return response

result = asyncio.run(run_workflow())
print(result)

/var/folders/5p/trf91_p17bg3d0t9l5pl9kdm0000gn/T/ipykernel_83703/881716511.py:15: UserWarning: Field name "result" in "ResponseEvent" shadows an attribute in parent "StopEvent"
  class ResponseEvent(StopEvent):


The poems suggest that success lies more in how we use the resources at our disposal rather than solely depending on external factors such as money or social prestige. They emphasize that each individual has sufficient materials for achievement, and what truly matters is the spirit or willpower one brings to the task.

For instance, "OPPORTUNITY" highlights that regardless of past failures or current setbacks, every morning offers new opportunities to succeed, with the poem's speaker rising to start anew daily. Similarly, in "WORK," the idea that work itself is a powerful force for personal development and success is underscored, encouraging individuals to embrace their potential by actively engaging in labor.

Overall, these poems teach that one should focus on proactive effort and persistent application rather than dwelling on past mistakes or waiting for external conditions to change.
